# Baseline Gender Biased Token Classifiers

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory ../data/token_clf_data/model_input/
* Multilabel classification
    * 3 categories of labels:
        1. Person Name: Unknown, Non-binary, Feminine, Masculine
        2. Linguistic: Generalization, Gendered Pronoun, Gendered Role
        3. Contextual: Empowering, Occupation, Omission, Stereotype
* Also try a [classifier chain](https://scikit-learn.org/stable/modules/multiclass.html#classifierchain)!!!

***

**Table of Contents**

[1.](#1) Logistic Regression

[2.](#2) Decision Tree

[3.](#3) Random Forest

[4.](#4) Error Analysis

***

Load necessary libraries:

In [1]:
# For custom functions and variables
import utils, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For visualization
import matplotlib.pyplot as plt

# For classifcation
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
# from sklearn.metrics import precision_recall_fscore_support

Load the train and validation (dev) data:

In [5]:
df_train = pd.read_csv(config.tokc_path+"model_input/token_train.csv", index_col=0)
df_dev = pd.read_csv(config.tokc_path+"model_input/token_validate.csv", index_col=0)
print(df_train.shape, df_dev.shape)
df_train.head()

(213094, 8) (70683, 8)


,description_id,sentence_id,ann_id,token_id,token,tag,field,subset
3,1,1,14384,7,The,B-Unknown,Title,train
4,1,1,24275,7,The,B-Masculine,Title,train
5,1,1,52952,7,The,B-Stereotype,Title,train
6,1,1,14384,8,Very,I-Unknown,Title,train
7,1,1,24275,8,Very,I-Masculine,Title,train


Binarize and encode the **training** data:

In [6]:
# Binarize and encode the data
# Reference: https://scikit-learn.org/stable/modules/preprocessing_targets.html#preprocessing-targets
# ---
# Reference: https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
df_train = df_train.drop(columns=["description_id","ann_id", "token_id", "field", "subset"])
df_dev = df_dev.drop(columns=["description_id","ann_id", "token_id", "field", "subset"])
v = DictVectorizer(sparse=False)

X_train = df_train.drop("tag", axis=1)
X_train = v.fit_transform(X_train.to_dict('records'))
y_train = df_train.tag.values
X_dev = df_dev.drop("tag", axis=1)
X_dev = v.fit_transform(X_dev.to_dict('records'))
y_dev = df_dev.tag.values

classes = np.unique(y_train)
classes = classes.to_list()

MemoryError: Unable to allocate 33.7 GiB for an array with shape (213094, 21223) and data type float64